In [23]:
from scipy.integrate import solve_ivp
import numpy as np
from scipy.integrate._ivp.ivp import OdeResult

### ODE solvers in Python HEM
The Python code uses [solve_ivp](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html).
 See [src/main.rs](src/main.rs) for Rust implementation. (WIP)

They are using two different *methods*:

- **RK45** - Runge-Kutta method of order 5(4) which is the default method
- **BDF** - Backward Differentiation Formula


The aim of this investigaion is the use a Rust library to replicate the same behaviour.
Example code later in this file showing usage.

Next steps:
- Investigate Rust libraries (ongoing).
- Understand the output from the method and see how this is used in Python. Do we need to match the exact method? Or can we get the same result with any method?

### SciPy's solve_ivp example usage

In [26]:
# example function from docs
def exponential_decay(t, y): return -0.5 * y # an example function

t_span = [0, 5]
y0 = [2] # scipy can handle multiple separate inputs here
# need to check if we're using this feature in Python HEM


# RK45
sol = solve_ivp(exponential_decay, t_span, y0) # RK45 by default

print("\n\nRK45 sol.t", sol.t)          # outputs:    [0.         0.11488132 1.26369452 3.06074656 4.81637262 5.        ]
print("RK45 sol.y", sol.y)              # outputs:    [[2.         1.88835583 1.0632438  0.43316531 0.18014905 0.16434549]]
print("RK45 y[0][-1]", sol.y[0][-1])    # outputs:    0.164345493039713


# BDF
sol = solve_ivp(exponential_decay, t_span, y0, method="BDF")

print("\nBDF sol.t", sol.t)                 # outputs: [0.00000000e+00 4.47325385e-03 8.94650770e-03 ... ]
print("BDF sol.y", sol.y)                   # outputs: [[[2.         1.99553517 1.99108007 1.94736925 ... ]]
print("BDF sol.y[:, -1]", sol.y[:, -1])     # outputs: [0.16416542]



RK45 sol.t [0.         0.11488132 1.26369452 3.06074656 4.81637262 5.        ]
RK45 sol.y [[2.         1.88835583 1.0632438  0.43316531 0.18014905 0.16434549]]
RK45 y[0][-1] 0.164345493039713

BDF sol.t [0.00000000e+00 4.47325385e-03 8.94650770e-03 5.36790462e-02
 9.84115847e-02 2.75295014e-01 4.52178444e-01 6.29061874e-01
 1.00228148e+00 1.37550108e+00 1.74872068e+00 2.12194029e+00
 2.62840119e+00 3.13486210e+00 3.64132301e+00 4.14778391e+00
 4.65424482e+00 5.00000000e+00]
BDF sol.y [[2.         1.99553517 1.99108007 1.94736925 1.9045969  1.7436898
  1.59600556 1.46073994 1.21185692 1.00567432 0.83468494 0.69273782
  0.53773684 0.41730645 0.32386407 0.25139098 0.19514591 0.16416542]]
BDF sol.y[:, -1] [0.16416542]


### Usage in Python HEM code
NOTE - these will error in this notebook because they reference code not copied over

### emitters.py

In [ ]:
# from emitters.py
temp_diff_emitter_rm_results = solve_ivp(
            func_temp_emitter_change_rate,
            (time_start, time_end),
            (temp_diff_start,),
            events=events,
            )

# later in the function (modified)
#temp_diff_emitter_rm_results.t_events[0][-1] and temp_diff_emitter_rm_results.y[0][-1] are used


time_temp_diff_max_reached = temp_diff_emitter_rm_results.t_events[0][-1]
temp_diff_emitter_rm_final = temp_diff_emitter_rm_results.y[0][-1]

# uses default method, which is RK45 (Runge-Kutta method of order 5(4))

### elec_storage_heater.py

In [ ]:
# from elec_storage_heater.py (modifed)\n",
sol: OdeResult = solve_ivp(fun=self.__func_core_temperature_change_rate(q_dis_modo=q_dis_modo)
            t_span=time_range
            y0=temp_core_and_wall
            method='BDF')

new_temp_core_and_wall: list = sol.y[: -1]

# uses BDF method - "Backward Differentiation Formula"